## Fine tuning the ResNet model

In this notebook, we will explore how to fine-tune a pre-trained model for image classification tasks. We will use the **ResNet-152x4** model, which is a deep convolutional neural network with *152* layers and *4* times more channels than the standard **ResNet-152**. This model was trained on the **ImageNet-21k** dataset, which is a large-scale dataset with about *14 million images and 21,000 classes*. We will fine-tune this model on different datasets that are commonly used for image classification, such as **ImageNet-1k**, **CIFAR-10**, **CIFAR-100**, **Oxford Pets** and **Oxford flowers**. We will save the results to compare them with the **ViT** later.

This notebook contains five datasets, you have to run the Utility Functions subsection then you can choose which datasets you want to evaluate the BiT model on by running its specific subsection. The following table contains the approximate running time for each dataset:

|    Dataset     | Time (hh:mm:ss) |
|:--------------:|:---------------:|
|    Imagenet    |    02:15:00     |
|    CIFAR-10    |    07:35:00     |
|   CIFAR-100    |    07:35:00     |
|  Oxford Pets   |    00:50:00     |
| Oxford Flowers |    00:40:00     |

**🚨 Note that this notebook will not run using the ResNet-152x4 model on colab free version 🚨**

------------------------------------------------------------------------

### Utility Functions

The next few cells show the functions used for fine-tuning the **ResNet-152x4** model. We start by importing the required modules:

In [ ]:
import os
import json
import time
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import OrderedDict
from torch.utils.data import random_split
from torchvision import transforms, datasets, models

------------------------------------------------------------------------

The function `get_res_loaders` is a helper function that can create the dataloaders for *training* and *testing* the **ResNet** model on different image classification datasets. The function takes two arguments: `dataset` and `batch_size`. The `dataset` argument is a string that specifies the name of the dataset to use, such as “imagenet”, “cifar10”, “cifar100”, “oxford_pets”, or “flowers_102”. The `batch_size` argument is an integer that specifies the number of images to feed to the model at each iteration. We can also adjust the `batch_size` argument according to the GPU we have. The function returns three dataloaders: one for the training data, one for the development data and one for the testing data. The development splits are as the following: 10% for Pets and Flowers, 2% for CIFAR.

The function uses a dictionary called `known_dataset_sizes` to store the image sizes for each dataset. It also uses a transformation function called `normalize_transform` to normalize the images using the channel means. The function then creates two different transformation functions for the training and testing data, using the PyTorch transforms module.

**🌀 Reproducibility Challenge: Image resolution**  
One challenge that we face in this notebook was the discrepancy between the image resolution used in the paper and the code. The paper reported using a resolution of **384x384** pixels for the ResNet models, but the code provided different resolutions for different datasets. In the code they used **128x128** for CIFAR datasets, **224x224** for Oxford datasets and **480x480** for ImageNet-1k dataset. To determine the best resolution, we would need to try all four resolutions, which is computationally expensive. However, this step was completed beforehand and it was found that the resolutions in `known_dataset_sizes` yield good results but may not be the optimal ones.

**Can you explain how using different resolutions affects computation and what other hyperparameters it may impact? 🤔**

**⚙️ Computation Challenge: Batch Size**  
The batch size used by the authors was **512** samples per batch which is considered huge, given that a 24GB GPU can hold the model and a maximum batch size of **16** samples for images with 128x128 resolution. Therefore, we can only use batch sizes that are compatible with our GPU memory.

In [ ]:
known_dataset_sizes = {
  'cifar10': (128, 128),
  'cifar100': (128, 128),
  'oxford_pets': (128, 128),
  'flowers_102': (128, 128),
  'imagenet': (384, 384),
}

def get_res_loaders(dataset="imagenet", batch_size=64):
    """
    This loads the whole dataset into memory and returns train and test data to
    be used by the ResNet model
    @param dataset (string): dataset name to load
    @param batch_size (int): batch size for training and testing

    @returns dict() with train and test data loaders with keys `train_loader`, `test_loader`
    """
    # Get image size
    crop = known_dataset_sizes[dataset]
    precrop = (160,160)
    # Normalization using channel means
    normalize_transform = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

    # Creating transform function
    train_transform = transforms.Compose([
                        transforms.Resize(precrop),
                        transforms.RandomCrop(crop),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        normalize_transform,
                    ])

    # Test transformation function
    test_transform =transforms.Compose([transforms.Resize(crop), transforms.ToTensor(), normalize_transform])

    # Loader arguments
    loader_args = {
        "batch_size": batch_size,
    }

     # Load the dataset from torchvision datasets
    if dataset == "imagenet":
        # Load the dataset
        original_test_dataset = datasets.ImageFolder(root=os.path.join('data', 'imagenet', 'val'), transform=test_transform)
        # Create Data Loader
        test_loader = torch.utils.data.DataLoader(
            dataset=original_test_dataset,
            shuffle=True,
            **loader_args)
        # Return Test Loader
        return {"test_loader": test_loader}

    elif dataset == "cifar10":
        # Load CIFAR-10
        original_train_dataset = datasets.CIFAR10(root=os.path.join('data', 'cifar10_data'),
                                             train=True, transform=train_transform, download=True)
        original_test_dataset = datasets.CIFAR10(root=os.path.join('data', 'cifar10_data'),
                                             train=False, transform=test_transform, download=True)
    elif dataset == "cifar100":
        # Load CIFAR-100
        original_train_dataset = datasets.CIFAR100(root=os.path.join('data', 'cifar100_data'),
                                             train=True, transform=train_transform, download=True)
        original_test_dataset = datasets.CIFAR100(root=os.path.join('data', 'cifar100_data'),
                                             train=False, transform=test_transform, download=True)
    elif dataset == "oxford_pets":
        # Load Oxford-IIIT Pets
        original_train_dataset = datasets.OxfordIIITPet(root=os.path.join('data', 'oxford_iiit_pets_data'),
                                             split='trainval', transform=train_transform, download=True)
        original_test_dataset = datasets.OxfordIIITPet(root=os.path.join('data', 'oxford_iiit_pets_data'),
                                             split='test', transform=test_transform, download=True)
    elif dataset == "flowers_102":
        # Load Oxford Flowers-102
        original_train_dataset = datasets.Flowers102(root=os.path.join('data', 'oxford_flowers_102_data'),
                                             split='train', transform=train_transform, download=True)
        original_test_dataset = datasets.Flowers102(root=os.path.join('data', 'oxford_flowers_102_data'),
                                             split='test', transform=test_transform, download=True)
    else:
        # Raise an error if the dataset is not valid
        raise ValueError("Invalid dataset name. Please choose one of the following: imagenet, cifar10, cifar100, oxford_pets, flowers_102")

    
    # Set the validation set size
    val_size = int(0.02 * len(original_train_dataset)) if dataset in ['cifar10', 'cifar100'] else int(0.1 * len(original_train_dataset))
    
    # Set train size as remaining data
    train_size = len(original_train_dataset) - val_size

    # Split the original train dataset into train and validation datasets
    train_dataset, val_dataset = random_split(original_train_dataset, [train_size, val_size])

    # Create the data loaders
    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset,
        shuffle=True,
        **loader_args)
    
    val_loader = torch.utils.data.DataLoader(
        dataset=val_dataset,
        shuffle=True,
        **loader_args)

    test_loader = torch.utils.data.DataLoader(
        dataset=original_test_dataset,
        shuffle=True,
        **loader_args)

    return {"train_loader": train_loader,
            "val_loader": val_loader,
            "test_loader": test_loader}

------------------------------------------------------------------------

Then we use the **ResNet** implementation from the [official repository](https://github.com/google-research/big_transfer/blob/master/bit_pytorch/models.py) to create our model.

In [ ]:
class StdConv2d(nn.Conv2d):
    """
    A custom convolutional layer that standardizes the weights before applying them.
    """
    def forward(self, x):
        w = self.weight
        # Compute the variance and mean of the weights along the channel, height and width dimensions
        v, m = torch.var_mean(w, dim=[1, 2, 3], keepdim=True, unbiased=False)
        # Standardize the weights by subtracting the mean and dividing by the standard deviation
        w = (w - m) / torch.sqrt(v + 1e-10)
        # Apply the standardized weights
        return F.conv2d(x, w, self.bias, self.stride, self.padding, self.dilation, self.groups)

# helper function to create a 3x3 convolutional layer with standardization.    
def conv3x3(cin, cout, stride=1, groups=1, bias=False):
    return StdConv2d(cin, cout, kernel_size=3, stride=stride, padding=1, bias=bias, groups=groups)

# helper function to create a 1x1 convolutional layer with standardization.
def conv1x1(cin, cout, stride=1, bias=False):
    return StdConv2d(cin, cout, kernel_size=1, stride=stride, padding=0, bias=bias)

# helper function to convert the weight format from TensorFlow (HWIO) to PyTorch (OIHW).
def tf2th(conv_weights):
    if conv_weights.ndim == 4:
        # Transpose the dimensions from height-width-input-output to output-input-height-width
        conv_weights = np.transpose(conv_weights, [3, 2, 0, 1])
    return torch.from_numpy(conv_weights)


class PreActBottleneck(nn.Module):
    """
    A custom residual block that uses pre-activation and group normalization.
    This is based on the paper "Identity Mappings in Deep Residual Networks" by Kaiming He et al.
    https://arxiv.org/abs/1603.05027
    However, this implementation differs from the original one by putting
    the stride on the 3x3 convolution instead of the 1x1 convolution.
    """
    def __init__(self, cin, cout=None, cmid=None, stride=1):
        super().__init__()
        # Set the output channels to be the same as the input channels if not specified
        cout = cout or cin
        # Set the middle channels to be one fourth of the output channels if not specified
        cmid = cmid or cout//4

        # Define the group normalization and convolution layers for the block
        self.gn1 = nn.GroupNorm(32, cin)
        self.conv1 = conv1x1(cin, cmid)
        self.gn2 = nn.GroupNorm(32, cmid)
        self.conv2 = conv3x3(cmid, cmid, stride)  # Original ResNetv2 has it on conv1!!
        self.gn3 = nn.GroupNorm(32, cmid)
        self.conv3 = conv1x1(cmid, cout)
        self.relu = nn.ReLU(inplace=True)

        # Define the projection layer for the residual branch if needed
        if (stride != 1 or cin != cout):
            self.downsample = conv1x1(cin, cout, stride)

    def forward(self, x):
        # Conv'ed branch
        out = self.relu(self.gn1(x))

        # Residual branch
        residual = x
        # If there is a projection layer, apply it to the output of the first activation
        if hasattr(self, 'downsample'):
            residual = self.downsample(out)

        # The first block has already applied pre-act before splitting, see Appendix.
        out = self.conv1(out)
        out = self.conv2(self.relu(self.gn2(out)))
        out = self.conv3(self.relu(self.gn3(out)))
        
        # Add the residual branch to the conv'ed branch and return
        return out + residual
    
    # helper function to load the weights from a TensorFlow model
    def load_from(self, weights, prefix=''):
        with torch.no_grad():
            # Copy the weights for each layer from the TensorFlow model to the PyTorch model
            self.conv1.weight.copy_(tf2th(weights[prefix + 'a/standardized_conv2d/kernel']))
            self.conv2.weight.copy_(tf2th(weights[prefix + 'b/standardized_conv2d/kernel']))
            self.conv3.weight.copy_(tf2th(weights[prefix + 'c/standardized_conv2d/kernel']))
            self.gn1.weight.copy_(tf2th(weights[prefix + 'a/group_norm/gamma']))
            self.gn2.weight.copy_(tf2th(weights[prefix + 'b/group_norm/gamma']))
            self.gn3.weight.copy_(tf2th(weights[prefix + 'c/group_norm/gamma']))
            self.gn1.bias.copy_(tf2th(weights[prefix + 'a/group_norm/beta']))
            self.gn2.bias.copy_(tf2th(weights[prefix + 'b/group_norm/beta']))
            self.gn3.bias.copy_(tf2th(weights[prefix + 'c/group_norm/beta']))
            # If there is a projection layer, copy its weights as well
            if hasattr(self, 'downsample'):
                self.downsample.weight.copy_(tf2th(weights[prefix + 'a/proj/standardized_conv2d/kernel']))
        
        # Return the PyTorch model with loaded weights
        return self

# ResNet Class
class ResNetV2(nn.Module):
    BLOCK_UNITS = {
        'r50': [3, 4, 6, 3],
        'r101': [3, 4, 23, 3],
        'r152': [3, 8, 36, 3],
    }

    def __init__(self, block_units, width_factor, head_size=21843, zero_head=False):
        super().__init__()
        wf = width_factor  # shortcut 'cause we'll use it a lot.

        self.root = nn.Sequential(OrderedDict([
            ('conv', StdConv2d(3, 64*wf, kernel_size=7, stride=2, padding=3, bias=False)),
            ('padp', nn.ConstantPad2d(1, 0)),
            ('pool', nn.MaxPool2d(kernel_size=3, stride=2, padding=0)),
            # The following is subtly not the same!
            #('pool', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))

        self.body = nn.Sequential(OrderedDict([
            ('block1', nn.Sequential(OrderedDict(
                [('unit01', PreActBottleneck(cin= 64*wf, cout=256*wf, cmid=64*wf))] +
                [(f'unit{i:02d}', PreActBottleneck(cin=256*wf, cout=256*wf, cmid=64*wf)) for i in range(2, block_units[0] + 1)],
            ))),
            ('block2', nn.Sequential(OrderedDict(
                [('unit01', PreActBottleneck(cin=256*wf, cout=512*wf, cmid=128*wf, stride=2))] +
                [(f'unit{i:02d}', PreActBottleneck(cin=512*wf, cout=512*wf, cmid=128*wf)) for i in range(2, block_units[1] + 1)],
            ))),
            ('block3', nn.Sequential(OrderedDict(
                [('unit01', PreActBottleneck(cin= 512*wf, cout=1024*wf, cmid=256*wf, stride=2))] +
                [(f'unit{i:02d}', PreActBottleneck(cin=1024*wf, cout=1024*wf, cmid=256*wf)) for i in range(2, block_units[2] + 1)],
            ))),
            ('block4', nn.Sequential(OrderedDict(
                [('unit01', PreActBottleneck(cin=1024*wf, cout=2048*wf, cmid=512*wf, stride=2))] +
                [(f'unit{i:02d}', PreActBottleneck(cin=2048*wf, cout=2048*wf, cmid=512*wf)) for i in range(2, block_units[3] + 1)],
            ))),
        ]))

        self.zero_head = zero_head
        self.head = nn.Sequential(OrderedDict([
            ('gn', nn.GroupNorm(32, 2048*wf)),
            ('relu', nn.ReLU(inplace=True)),
            ('avg', nn.AdaptiveAvgPool2d(output_size=1)),
            ('conv', nn.Conv2d(2048*wf, head_size, kernel_size=1, bias=True)),
        ]))

    def forward(self, x):
        x = self.head(self.body(self.root(x)))
        assert x.shape[-2:] == (1, 1)  # We should have no spatial shape left.
        return x[...,0,0]

    def load_from(self, weights, prefix='resnet/'):
        with torch.no_grad():
            self.root.conv.weight.copy_(tf2th(weights[f'{prefix}root_block/standardized_conv2d/kernel']))
            self.head.gn.weight.copy_(tf2th(weights[f'{prefix}group_norm/gamma']))
            self.head.gn.bias.copy_(tf2th(weights[f'{prefix}group_norm/beta']))
            if self.zero_head:
                nn.init.zeros_(self.head.conv.weight)
                nn.init.zeros_(self.head.conv.bias)
            else:
                self.head.conv.weight.copy_(tf2th(weights[f'{prefix}head/conv2d/kernel']))
                self.head.conv.bias.copy_(tf2th(weights[f'{prefix}head/conv2d/bias']))

            for bname, block in self.body.named_children():
                for uname, unit in block.named_children():
                    unit.load_from(weights, prefix=f'{prefix}{bname}/{uname}/')
        return self

------------------------------------------------------------------------

The following are two helper functions that we use for calculating and evaluating the model performance:

-   `get_accuracy`: This function takes the model predictions and the true labels as inputs and returns the accuracy as a float value.
-   `evaluate_on_test`: This function takes the model, the test dataloader, and the device as inputs and returns the test accuracy of the model on the given test set.

In [ ]:
# Function takes predictions and true values to return accuracies
def get_accuracy(logit, true_y):
    pred_y = torch.argmax(logit, dim=1)
    return (pred_y == true_y).float().mean()

# This Function is used to evaluate the model
def evaluate_on_test(model, test_loader, device="cpu"):
    # Evaluate the model on all the test batches
    accuracies = []
    model.eval()
    for batch_idx, (data_x, data_y) in enumerate(test_loader):
        data_x = data_x.to(device)
        data_y = data_y.to(device)

        model_y = model(data_x)
        batch_accuracy = get_accuracy(model_y, data_y)

        accuracies.append(batch_accuracy.item())

        if batch_idx % 1000 == 0:
            print(f"Mean accuracy at batch: {batch_idx} is {np.mean(accuracies) * 100}")

    test_accuracy = np.mean(accuracies) * 100
    print(f"Test accuracy: {test_accuracy}")
    return test_accuracy

------------------------------------------------------------------------

The function `train_res_model` takes the following arguments and returns the train and test accuracies of the model:

-   `loaders`: A dictionary of PyTorch dataloaders for the train and test sets.
-   `model_name`: A string to specify the name of the ResNet model to use. The default is ‘r152’, which corresponds to ResNet-152.
-   `bit_model`: A string to specify the name of the pre-trained Big Transfer (BiT) model to use. The default is “BiT-M-R152x4”, which corresponds to BiT-M with ResNet-152 backbone and width factor 4.
-   `width_factor`: An integer to scale the width of the ResNet model. The default is 4, which means the number of channels in each layer is multiplied by 4.
-   `lr`: A float to set the learning rate for the optimizer. The default is 0.0003.
-   `epochs`: An integer to set the number of epochs for the training loop. The default is 10.

The function first creates a ResNet model with the specified arguments and loads the weights from the pre-trained BiT model. Then, it creates an optimizer and a scheduler for the training process. Finally, it runs a training loop for the given number of epochs, where it computes the loss and accuracy for each batch of data, updates the model parameters, and adjusts the learning rate. Then it passes the model and the test loader to the `evaluate_on_test` function and then returns the test accuracy

**🌀 Reproducibility Challenge: learning rate**  
The second reproducibility challenge that we encountered in this notebook was the lack of information about the optimal learning rate for each dataset. The paper only reported the learning rates used for the grid search, but not the ones selected for the final experiments. As a result, we assumed that the learning rates we use for each dataset would yield good results, but not necessarily the best.

**Based on the running time table provided above, approximately how long would it take to run the grid search for the CIFAR-10 dataset? 🧐**

**⚙️ Computation Challenge: number of epochs**  
Another challenge that we faced in this notebook was the ambiguity of the number of epochs used for fine-tuning. The paper did not specify the exact number of epochs, but only the number of steps relative to a batch size of 512 (which are huge). However, in some parts of the paper, the results were reported based on epochs. This made it difficult to compare our results with the paper’s results. Therefore, we decided to experiment with different numbers of epochs to find the best one for each dataset.

**Based on the table provided in the Experiments notebook, how many epochs would be equivalent to the number of steps used by the authors on the CIFAR-100 dataset (50,000 training samples) if they used a batch size of 512 as stated in the paper? And how much time would this take? 🤔**

In [ ]:
# Function to train the model and return train and test accuracies
def train_res_model(loaders, title='', model_name='r152', bit_model="BiT-M-R152x4",
                       width_factor=4, lr=0.0003, epochs=10, random_seed=42, save=False):

    # Create experiment directory name if none
    experiment_dir = os.path.join('experiments', title)

    # make experiment directory
    os.makedirs(experiment_dir, exist_ok=True)

    # Set the seed
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)

    # Check if GPU is available
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
        print("CUDA Recognized")
    else:
        device = torch.device('cpu')

    # Get num_classes
    num_classes = len(loaders["train_loader"].dataset.dataset.classes)

    # Get weights
    print(f"Loading {bit_model} weights...")
    weights = get_weights(bit_model)
    print("Weight successfully loaded")

    # Initialize the ResNet model
    model = ResNetV2(ResNetV2.BLOCK_UNITS[model_name], width_factor=width_factor, head_size=num_classes, zero_head=True)
    model.load_from(weights)
    model.to(device);

    # Create the optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Calculate per_step, authors used 512 batch size
    batch_size = loaders["train_loader"].batch_size
    T_max = len(loaders["train_loader"]) * epochs

    # Create the scheduler with cosine learning rate decay
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_max)

    # Create the loss function
    criterion = torch.nn.CrossEntropyLoss()

    # Iterate over the number of epochs
    for epoch in range(1, epochs + 1):
        # Make model params trainable
        model.to(device);
        model.train()
        print(f"Epoch {epoch}")
        accuracies = []
        losses = []

        # Calculate loss and gradients for models on every training batch
        for batch_idx, (data_x, data_y) in enumerate(loaders["train_loader"]):
            data_x = data_x.to(device)
            data_y = data_y.to(device)

            optimizer.zero_grad()
            model_y = model(data_x)
            loss = criterion(model_y, data_y)
            batch_accuracy = get_accuracy(model_y, data_y)

            # Perform back propagation
            loss.backward()
            optimizer.step()

            accuracies.append(batch_accuracy.item())
            losses.append(loss.item())

            scheduler.step()

        # Store training accuracy for plotting
        train_loss = np.mean(losses)
        train_accuracy = np.mean(accuracies)*100
        print("Train accuracy: {} Train loss: {}".format(train_accuracy, train_loss))

        # Evaluate the model on all the test batches
        accuracies = []
        losses = []
        model.eval()
        # Move the model to CPU
        model.to("cpu")
        for batch_idx, (data_x, data_y) in enumerate(loaders["val_loader"]):
            # Move the data to CPU
            data_x = data_x.to("cpu")
            data_y = data_y.to("cpu")

            model_y = model(data_x)
            loss = criterion(model_y, data_y)
            batch_accuracy = get_accuracy(model_y, data_y)

            accuracies.append(batch_accuracy.item())
            losses.append(loss.item())

        # Store test accuracy for plotting
        val_loss = np.mean(losses)
        val_accuracy = np.mean(accuracies)*100
        print("Validation accuracy: {} Validation loss: {}".format(val_accuracy, val_loss))

    if save:
        # Save the final model
        torch.save({
            'model': model.state_dict()
        }, os.path.join(experiment_dir, f'{bit_model}.pt'))

    # Evaluate on test set
    test_acc = evaluate_on_test(model, loaders['test_loader'])

    # Delete the data and model outputs from GPU memory
    del model
    # Release unused memory
    torch.cuda.empty_cache()

    # return the accuracies
    return test_acc

------------------------------------------------------------------------

The following function `plot_images_from_dataloader` takes a PyTorch dataloader as an input and plots 10 of the images from the first batch of data. The function also shows the labels of the images according to the classes attribute of the dataloader’s dataset

In [ ]:
# Define a function to plot 10 of the images
def plot_images_from_dataloader(dataloader, classes=None):
    # Initialize empty tensors for images and labels
    images = torch.empty(0)
    labels = torch.empty(0, dtype=torch.long)

    # Loop until the images and labels have at least 10 elements
    while len(images) < 10:
        # Get the next batch of images and labels from the dataloader
        batch_images, batch_labels = next(iter(dataloader))
        # Concatenate the batch images and labels to the existing tensors
        images = torch.cat((images, batch_images), dim=0)
        labels = torch.cat((labels, batch_labels), dim=0)
    # Get class names
    if classes is None:
        classes = dataloader.dataset.dataset.classes
    # Create a figure with 2 rows and 5 columns
    fig, axes = plt.subplots(2, 5, figsize=(10, 4))
    for i, ax in enumerate(axes.flat):
        image = images[i]
        label = classes[labels[i]]
        # Unnormalize the image
        image = image / 2 + 0.5
        image = image.numpy()
        # Transpose the image
        image = np.transpose(image, (1, 2, 0))
        # Plot the image on the axis
        ax.imshow(image)
        # Set title as label
        ax.set_title(label)
        # Turn off the axis ticks
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

------------------------------------------------------------------------

The following cell contains function to format time and print it, while the other function is used to load the downloaded model weights.

In [ ]:
# Function used to load npz files
def get_weights(bit_variant):
    return np.load(f'pretrained_models/{bit_variant}.npz')

# Function used to calculated time
def print_time(start_time, end_time):
    # Calculate the difference in seconds
    diff = end_time - start_time

    # Convert the difference to hours, minutes, and seconds
    hours, remainder = divmod(diff, 3600)
    minutes, seconds = divmod(remainder, 60)

    # Create time in hours:minutes:seconds format
    time_string = f"{int(hours)}:{int(minutes)}:{seconds}"

    # Print the time
    print(f"Cell execution time: {time_string}")

    return time_string

------------------------------------------------------------------------

We download the models that we will be using through the rest of the notebook:

-   The `BiT-M-R152x4` is pre-trained on the **ImageNet-21k** and not fine tuned at all.
-   The `BiT-M-R152x4-ILSVRC2012` is pre-trained on the **ImageNet-21k** and fine tuned on the **ImageNet-1k**

In [ ]:
# Create a directory to hold the pre-trained models
!mkdir -p pretrained_models

# Download the ResNet152x4 model
![ -e pretrained_models/BiT-M-R152x4.npz ] || \
curl -L -o pretrained_models/BiT-M-R152x4.npz "https://storage.googleapis.com/bit_models/BiT-M-R152x4.npz"

# Download the ResNet152x4 model fine tuned on ImageNet
![ -e pretrained_models/BiT-M-R152x4-ILSVRC2012.npz ] || \
curl -L -o pretrained_models/BiT-M-R152x4-ILSVRC2012.npz "https://storage.googleapis.com/bit_models/BiT-M-R152x4-ILSVRC2012.npz"

------------------------------------------------------------------------

**⚙️ Computation Challenge: High GPU memory requirement**  
After installing the models from the previous cell, you can see that the model itself is 4GB which means it will take a huge space in the GPU. This actually challenging as you will want to optimize the GPU RAM without crashing it.

The batch size chosen in the following runs are for a `RTX6000 GPU`. If you are using another GPU you can try changing it to maximize the usage of your GPU.

Note that maximizing GPU usage is just an option, we did this as authors used a very high batch size. you can try different batch sizes to find which batch size gives the best performance.

You can use any of these commands in the terminal while running the cells to check how much of your GPU memory is utilized:

-   `nvidia-smi --query-gpu=gpu_name,memory.used,memory.free,memory.total --format=csv`
-   `nvidia-smi pmon -s m -c 1`

------------------------------------------------------------------------

### ImageNet

The ImageNet dataset consists of **1000** object classes and contains **1,281,167** training images, **50,000** validation images and **100,000** test images. The images vary in resolution but it is common practice to train deep learning models on sub-sampled images of **256x256** pixels. This dataset is widely used for image classification and localization tasks and has been the benchmark for many state-of-the-art algorithms.

------------------------------------------------------------------------

The first step is to load the dataset and choose the `batch_size` that suits our GPU capacity. This will help us avoid errors during the training process. Next, we use the `plot_images_from_dataloader` function to display 10 of the images from the first batch of data. We can see the labels of the images on top of each plot. However, we need to make sure that the batch size is at least 10, otherwise the function will raise an error.

In [ ]:
# Plot some images from the ImageNet dataset
loader = get_res_loaders(dataset="imagenet", batch_size=1)
plot_images_from_dataloader(loader["test_loader"], loader["test_loader"].dataset.classes)

------------------------------------------------------------------------

We can use the `BiT-M-R152x4-ILSVRC2012` model, which is already fine-tuned on the **ImageNet** dataset, to evaluate its performance on the test set. This model is provided by the authors of the paper, so we expect to reproduce their results exactly. We will create the model object, load its pre-trained weights, and run it on the test data.

In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print("CUDA Recognized")
else:
    device = torch.device('cpu')

# Get num_classes
num_classes = len(loader["test_loader"].dataset.classes)

# Get weights
print(f"Loading BiT-M-R152x4-ILSVRC2012 weights...")
weights = get_weights("BiT-M-R152x4-ILSVRC2012")
print("Weight successfully loaded")

# Initialize the ResNet model
model = ResNetV2(ResNetV2.BLOCK_UNITS['r152'], width_factor=4, head_size=num_classes)
model.load_from(weights)
model.to(device);

------------------------------------------------------------------------

We use the `evaluate_on_test` function to get the test accuracy for the model. The validation data that we use for evaluation is of size 50k samples. This means that it is normal for the evaluation to take some time. We will print the time taken to evaluate the model and then delete the model to empty the GPU memory for the next dataset.

**Please note that we are calculating the time it takes to evaluate the model on the validation data 🕑**

In [ ]:
# Save start time
start_time = time.time()
# Print the Performance of the Ready fine tuned model
test_acc_imagenet = evaluate_on_test(model, loader["test_loader"], device)
# Calculate and print cell execution time
end_time = time.time()
imagenet_time = print_time(start_time, end_time)
# delete model to free gpu
del model
# Release unused memory
torch.cuda.empty_cache()

------------------------------------------------------------------------

We store the model’s performance metrics in a dictionary, which we will use to create a table later.

🛑 If you want to try a different model, please remember to change the name of the json file where you save the dictionary, so that you do not **overwrite** your previous results.

In [ ]:
# Create dictionary runs
runs = {}

# Check if the file exists
if os.path.exists("experiments/resnet.json"):
    # Open the file in read mode
    with open("experiments/resnet.json", "r") as f:
        # Load the data from the file to runs
        runs = json.load(f)

# Add the results to a dictionary
runs["imagenet"] =  test_acc_imagenet

# Create dictionary times
times = {}

# Check if the file exists
if os.path.exists("experiments/resnet_time.json"):
    # Open the file in read mode
    with open("experiments/resnet_time.json", "r") as f:
        # Load the data from the file to times
        times = json.load(f)

# Add the time to a dictionary
times["imagenet"] =  imagenet_time

In [ ]:
# Save the outputs in a json file
with open("experiments/resnet.json", "w") as f:
    json.dump(runs, f)

with open("experiments/resnet_time.json", "w") as f:
    json.dump(times, f)

------------------------------------------------------------------------

### CIFAR-10

The CIFAR-10 dataset consists of **60,000 32x32** color images in **10** different classes. The 10 classes are airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck. There are **6,000** images per class, with **5,000** for training and **1,000** for testing. It is a popular benchmark for image classification and deep learning research.

------------------------------------------------------------------------

We start by loading the **CIFAR-10** dataset and some of the images in it. Notice that the *batch size* increased as the resolution of the image decreased.

In [ ]:
# Plot some images from the CIFAR-10 dataset
loader = get_res_loaders(dataset="cifar10", batch_size=16)
plot_images_from_dataloader(loader["train_loader"])

------------------------------------------------------------------------

We then fine-tune the model for 10 epochs on the dataset and get the train and test accuracies. As mentioned above the dataset has 50k samples for training, that means it will take some time to run the below cell.

**Were you able to guess the number of epochs correctly based on your answer above? 🤔**

In [ ]:
start_time = time.time()
# Fine tune the model on CIFAR-10
test_acc_cifar10 = train_res_model(loaders=loader)
# Calculate and print cell execution time
end_time = time.time()
cifar10_time = print_time(start_time, end_time)

------------------------------------------------------------------------

We save the results in the same dictionary as before.

In [ ]:
# Create dictionary runs
runs = {}

# Check if the file exists
if os.path.exists("experiments/resnet.json"):
    # Open the file in read mode
    with open("experiments/resnet.json", "r") as f:
        # Load the data from the file to runs
        runs = json.load(f)

# Add the results to a dictionary
runs["cifar10"] = test_acc_cifar10

# Create dictionary times
times = {}

# Check if the file exists
if os.path.exists("experiments/resnet_time.json"):
    # Open the file in read mode
    with open("experiments/resnet_time.json", "r") as f:
        # Load the data from the file to times
        times = json.load(f)

# Add the time to a dictionary
times["cifar10"] =  cifar10_time

In [ ]:
# Save the outputs in a json file
with open("experiments/resnet.json", "w") as f:
    json.dump(runs, f)

with open("experiments/resnet_time.json", "w") as f:
    json.dump(times, f)

------------------------------------------------------------------------

### CIFAR-100

The CIFAR-100 dataset consists of **60,000 32x32** color images in **100** different classes. The 100 classes are grouped into 20 superclasses, such as aquatic mammals, flowers, insects, vehicles, etc. There are **600** images per class, with **500** for training and **100** for testing. It is also a commonly benchmark for image classification and deep learning research.

------------------------------------------------------------------------

As before we load and plot the dataset that we will fine tune the model on.

In [ ]:
# Plot some images from the CIFAR-100 dataset
loader = get_res_loaders(dataset="cifar100", batch_size=16)
plot_images_from_dataloader(loader["train_loader"])

------------------------------------------------------------------------

We fine-tune the model again for 10 epochs on the **CIFAR-100** dataset.

In [ ]:
start_time = time.time()
# Fine tune the model on CIFAR-100
test_acc_cifar100 = train_res_model(loaders=loader)
# Calculate and print cell execution time
end_time = time.time()
cifar100_time = print_time(start_time, end_time)

------------------------------------------------------------------------

We save the results to be able to use it later for creating this model’s results table.

In [ ]:
# Create dictionary runs
runs = {}

# Check if the file exists
if os.path.exists("experiments/resnet.json"):
    # Open the file in read mode
    with open("experiments/resnet.json", "r") as f:
        # Load the data from the file to runs
        runs = json.load(f)

# Add the results to a dictionary
runs["cifar100"] = test_acc_cifar100

# Create dictionary times
times = {}

# Check if the file exists
if os.path.exists("experiments/resnet_time.json"):
    # Open the file in read mode
    with open("experiments/resnet_time.json", "r") as f:
        # Load the data from the file to times
        times = json.load(f)

# Add the time to a dictionary
times["cifar100"] =  cifar100_time

In [ ]:
# Save the outputs in a json file
with open("experiments/resnet.json", "w") as f:
    json.dump(runs, f)

with open("experiments/resnet_time.json", "w") as f:
    json.dump(times, f)

------------------------------------------------------------------------

### Oxford-IIIT Pets

The Oxford-IIIT Pets is a **37** category pet dataset with roughly **200** images for each class created by the Visual Geometry Group at Oxford. The images have large variations in scale, pose and lighting. All images have an associated ground truth annotation of breed, head ROI (region of interest), and pixel level trimap segmentation. The dataset is useful for fine-grained image classification and segmentation tasks.

------------------------------------------------------------------------

We start again by loading and plotting the dataset. Make sure the batch size written is suitable for the GPU you are using to prevent any errors.

In [ ]:
# Plot some images from the Oxford-IIIT Pets dataset
loader = get_res_loaders(dataset="oxford_pets", batch_size=16)
plot_images_from_dataloader(loader["train_loader"])

------------------------------------------------------------------------

We fine-tune the model pre-trained on `ImageNet-21k` on the `OxfordPets` dataset. We get the training and testing accuracies for 10 epochs in the `train_acc_oxford_pets` and `test_acc_oxford_pets` arrays.

In [ ]:
start_time = time.time()
# Fine tune the model on Oxford-IIIT Pets
test_acc_oxford_pets = train_res_model(loaders=loader, lr=0.0001)
# Calculate and print cell execution time
end_time = time.time()
oxford_pets_time = print_time(start_time, end_time)

------------------------------------------------------------------------

We save the results in the runs dictionary under the dataset name.

In [ ]:
# Create dictionary runs
runs = {}

# Check if the file exists
if os.path.exists("experiments/resnet.json"):
    # Open the file in read mode
    with open("experiments/resnet.json", "r") as f:
        # Load the data from the file to runs
        runs = json.load(f)

# Add the results to a dictionary
runs["oxford_pets"] = test_acc_oxford_pets

# Create dictionary times
times = {}

# Check if the file exists
if os.path.exists("experiments/resnet_time.json"):
    # Open the file in read mode
    with open("experiments/resnet_time.json", "r") as f:
        # Load the data from the file to times
        times = json.load(f)

# Add the time to a dictionary
times["oxford_pets"] =  oxford_pets_time

In [ ]:
# Save the outputs in a json file
with open("experiments/resnet.json", "w") as f:
    json.dump(runs, f)

with open("experiments/resnet_time.json", "w") as f:
    json.dump(times, f)

------------------------------------------------------------------------

### Oxford Flowers-102

The Oxford Flowers-102 dataset consists of **102** flower categories commonly occurring in the United Kingdom. Each class consists of between **40 and 258** images. The images have large scale, pose and light variations. In addition, there are categories that have large variations within the category and several very similar categories. The dataset also provides image labels, segmentations, and distances based on shape and color features.

------------------------------------------------------------------------

The PyTorch dataset for this dataset does not contain the class labels, so we create an array called `flower_classes` to store them. We then use dataloaders to load the dataset and display the first 10 images from the train loader.

In [ ]:
# Plot some images from the Oxford Flowers-102 Pets dataset
loader = get_res_loaders(dataset="flowers_102", batch_size=16)

# We initialize the flowers names as they are not on Pytorch (used for plotting)
flower_classes = ['pink primrose', 'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',
 'english marigold', 'tiger lily', 'moon orchid', 'bird of paradise', 'monkshood', 'globe thistle',
 'snapdragon', "colt's foot", 'king protea', 'spear thistle', 'yellow iris', 'globe-flower', 'purple coneflower',
 'peruvian lily', 'balloon flower', 'giant white arum lily', 'fire lily', 'pincushion flower', 'fritillary',
 'red ginger', 'grape hyacinth', 'corn poppy', 'prince of wales feathers', 'stemless gentian', 'artichoke',
 'sweet william', 'carnation', 'garden phlox', 'love in the mist', 'mexican aster', 'alpine sea holly',
 'ruby-lipped cattleya', 'cape flower', 'great masterwort', 'siam tulip', 'lenten rose', 'barbeton daisy',
 'daffodil', 'sword lily', 'poinsettia', 'bolero deep blue', 'wallflower', 'marigold', 'buttercup', 'oxeye daisy',
 'common dandelion', 'petunia', 'wild pansy', 'primula', 'sunflower', 'pelargonium', 'bishop of llandaff', 'gaura',
 'geranium', 'orange dahlia', 'pink-yellow dahlia', 'cautleya spicata', 'japanese anemone', 'black-eyed susan',
 'silverbush', 'californian poppy', 'osteospermum', 'spring crocus', 'bearded iris', 'windflower', 'tree poppy',
 'gazania', 'azalea', 'water lily', 'rose', 'thorn apple', 'morning glory', 'passion flower', 'lotus', 'toad lily',
 'anthurium', 'frangipani', 'clematis', 'hibiscus', 'columbine', 'desert-rose', 'tree mallow', 'magnolia',
 'cyclamen', 'watercress', 'canna lily', 'hippeastrum', 'bee balm', 'ball moss', 'foxglove', 'bougainvillea',
 'camellia', 'mallow', 'mexican petunia', 'bromelia', 'blanket flower', 'trumpet creeper', 'blackberry lily']

# Save the Class names in the dataset
loader["train_loader"].dataset.dataset.classes = flower_classes
# Plot dataset
plot_images_from_dataloader(loader["train_loader"])

------------------------------------------------------------------------

We fine-tune the model on the dataset and obtain the train and test accuracies array. We change the number of epochs to 14 for this dataset as the size of the dataset is smaller than the other datasets. We also change the learning to `0.0001` as the fine-tuning yields better results with this learning rate.

In [ ]:
start_time = time.time()
# Fine tune the model on Oxford Flowers-102
test_acc_oxford_flowers = train_res_model(loader, epochs=14, lr=0.0001)
# Calculate and print cell execution time
end_time = time.time()
oxford_flowers_time = print_time(start_time, end_time)

------------------------------------------------------------------------

We store the result in `runs` dictionary to be used later for creating the table.

In [ ]:
# Create dictionary runs
runs = {}

# Check if the file exists
if os.path.exists("experiments/resnet.json"):
    # Open the file in read mode
    with open("experiments/resnet.json", "r") as f:
        # Load the data from the file to runs
        runs = json.load(f)

runs["oxford_flowers"] = test_acc_oxford_flowers

# Create dictionary times
times = {}

# Check if the file exists
if os.path.exists("experiments/resnet_time.json"):
    # Open the file in read mode
    with open("experiments/resnet_time.json", "r") as f:
        # Load the data from the file to times
        times = json.load(f)

# Add the time to a dictionary
times["oxford_flowers"] =  oxford_flowers_time

In [ ]:
# Save the outputs in a json file
with open("experiments/resnet.json", "w") as f:
    json.dump(runs, f)

with open("experiments/resnet_time.json", "w") as f:
    json.dump(times, f)

------------------------------------------------------------------------

**Can you explain the differences between our approach to fine-tuning the model and the approach described in the paper? 🧐**

You don’t need to check the paper as everything they did was explained in this notebook and the table provided in the experiments notebook.

------------------------------------------------------------------------

Although the authors did not provide results for the fine-tuned **ResNet** model pre-trained on the **ImageNet-21k** dataset in this paper, we can compare our results to their results for the model pre-trained on the proprietary **JFT-300M** dataset. Please fill in the following table template:

|       Model        | Imagenet | CIFAR-10 | CIFAR-100 | Oxford Pets | Oxford Flowers |
|:------------------:|:-------:|:-------:|:--------:|:----------:|:-------------:|
| Authors’ BiT (JFT) |          |          |           |             |                |
|   Our BiT (I21k)   |          |          |           |             |                |
|     Difference     |          |          |           |             |                |

**Given the difference in the size of the pre-training datasets, do these results seem reasonable to you? In your opinion, are these results satisfactory, or could we have tried something else to achieve better results? 🤔**

------------------------------------------------------------------------

Now that we are done with the **ResNet** model which we consider our baseline, we can start fine-tuning the the **ViT** model.